# Building a Bioinformatics Workflow with Jupyter & Anaconda

## Variant Calling Workflow

Burke Squires

https://github.com/burkesquires

---

#### Scenario:

Have yeast NGS data and we want to detect variants from the data

[Source: http://www.htslib.org/workflow/](http://www.htslib.org/workflow/)

Software that is need for pipeline:

- `bwa`
- `samtools`
- `bcftools`

Samtools is a suite of programs for interacting with high-throughput sequencing data. It consists of three separate repositories:

- Samtools: Reading/writing/editing/indexing/viewing SAM/BAM/CRAM format
- BCFtools: Reading/writing BCF2/VCF/gVCF files and calling/filtering/summarising SNP and short indel sequence variants
- HTSlib: A C library for reading/writing high-throughput sequencing data

Samtools and BCFtools both use HTSlib internally, but these source packages contain their own copies of htslib so they can be built independently.

## Download some data:

In [ ]:
!curl ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR507/SRR507778/SRR507778_1.fastq.gz | gzip -d | head -100000 > y1.fastq
!curl ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR507/SRR507778/SRR507778_2.fastq.gz | gzip -d | head -100000 > y2.fastq
!curl ftp://ftp.ensembl.org/pub/current_fasta/saccharomyces_cerevisiae/dna/Saccharomyces_cerevisiae.R64-1-1.dna_sm.toplevel.fa.gz | gzip -d > yeast.fasta

---

## Prepare the BWA indices

We need to ensure there exists a .fai fasta index and also indices for whichever aligner we are using (Bwa-mem in this example).

    samtools faidx yeast.fasta
    bwa index yeast.fasta

In [ ]:
!samtools faidx yeast.fasta
!bwa index yeast.fasta

---

In [ ]:
!ls

## Produce the Alignments

The aligner is likely to output SAM in the same order or similar order to the input fastq files. It won’t be outputting in chromosome position order, so the output is typically not well suited to CRAM.

    bwa mem -R '@RG\tID:foo\tSM:bar\tLB:library1' yeast.fasta y1.fastq y2.fastq > yeast.sam

The -R option adds a read-group line and applies that read-group to all aligned sequence records. It is not necessary, but a recommended practice.

In [ ]:
!bwa mem -R '@RG\tID:foo\tSM:bar\tLB:library1' yeast.fasta y1.fastq y2.fastq > yeast.sam

---

## Sort into chromosome/positon order

Ideally at this point we would be outputting CRAM directly, but at present samtools 1.0 does not have a way to indicate the reference on the command line. We can output to BAM instead and convert (below), or modify the SAM @SQ header to include MD5 sums in the M5: field.

    samtools sort -O bam -T /tmp -l 0 -o yeast.bam yeast.sam

The “-l 0” indicates to use no compression in the BAM file, as it is transitory and will be replaced by CRAM soon. We may wish to use -l 1 if disk space is short and we wish to reduce temporary file size.

In [ ]:
!samtools sort -O bam -T /tmp -l 0 -o yeast.bam yeast.sam

---

## Convert to CRAM format

    samtools view -T yeast.fasta -C -o yeast.cram yeast.bam

Note that since the BAM file did not have M5 tags for the reference sequences, they are computed by Samtools and added to the CRAM. In a production environment, this step can be avoided by ensuring that the M5 tags are already in the SAM/BAM header.

In [ ]:
!samtools view -T yeast.fasta -C -o yeast.cram yeast.bam

__Using CRAM within Samtools__

CRAM is primarily a reference-based compressed format, meaning that only differences between the stored sequences and the reference are stored.

For a workflow this has a few fundamental effects:

Alignments should be kept in chromosome/position sort order.
The reference must be available at all times. Losing it may be equivalent to losing all your read sequences.
Technically CRAM can work with other orders but it can become inefficient due to a large amount of random access across the reference genome. The current implementation of CRAM in htslib 1.0 is also inefficient in size for unsorted data, although this will be rectified in upcoming releases.

In CRAM format the reference sequence is linked to by the md5sum (M5 auxiliary tag) in the CRAM header (@SQ tags). This is mandatory and part of the CRAM specification. In SAM/BAM format, these M5 tags are optional. Therefore converting from SAM/BAM to CRAM requires some additional overhead to link the CRAM to the correct reference sequence.



The last 3 steps can be combined into a pipeline to reduce disk I/O:

    bwa mem yeast.fasta y1.fastq y2.fastq | \
    samtools sort -O bam -l 0 -T /tmp - | \
    samtools view -T yeast.fasta -C -o yeast.cram -

In [ ]:
!bwa mem yeast.fasta y1.fastq y2.fastq | \
samtools sort -O bam -l 0 -T /tmp - | \
samtools view -T yeast.fasta -C -o yeast.cram -

---

## Viewing in alignment and pileup format

See the variant calling workflow for more advanced examples.

    samtools view yeast.cram
    samtools mpileup -f yeast.fasta yeast.cram

In [ ]:
!samtools view yeast.cram > yeast.sam

In [ ]:
!samtools mpileup -f yeast.fasta yeast.cram > yeast_variants.pileup

---

## View (some  of) our results

    head -n 25 yeast.sam
    head -n 25 yeast_variants.pileup

In [ ]:
!head -n 25 yeast.sam

In [ ]:
!head -n 25 yeast_variants.pileup

---

#### Glossary of file formats

https://www.ncbi.nlm.nih.gov/sra/docs/submitformats/

Sequence data formats:
- FASTA: Simple format for DNA or peptide sequences.
- FASTQ: Stores sequences and sequence __quality__ information together.

Alignment data formats
- SAM / BAM Sequence Alignment/Map

Variation data
- VCF / BCF Variant Call Format